### Importar TensorFlow 2.0  y otras librerias

In [1]:
#!pip install tensorflow-gpu==2.0.0-alpha0

# from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

print(tf.__version__)

import numpy as np
import os
import time

2.6.0


### Descarga del conjunto de datos

In [2]:
path_to_fileDL = tf.keras.utils.get_file('DL-Introduccion-practica-con-Keras-1aParte.txt', 'https://raw.githubusercontent.com/jorditorresBCN/Deep-Learning-Introduccion-practica-con-Keras/master/DeepLearning-Introduccion-practica-con-Keras-PRIMERA-PARTE.txt')  

#path_to_fileDL = tf.keras.utils.get_file('Shakespear.txt', 'https://cs.stanford.edu/people/karpathy/char-rnn/shakespear.txt')

text = open(path_to_fileDL, 'rb').read().decode(encoding='utf-8')
print('Longitud del texto:        {} carácteres'.format(len(text)))

vocab = sorted(set(text))

print ('El texto está compuesto de estos {} carácteres:'.format(len(vocab)))
print (vocab)

212992/203286 [===============================] - 0s 1us/step
Longitud del texto:        203251 carácteres
El texto está compuesto de estos 92 carácteres:
['\n', '\r', ' ', '!', '"', '#', '%', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\xad', 'ÿ', 'Š', '‡', '…']


In [3]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

for char,_ in zip(char2idx, range(len(vocab))):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))

  '\n':   0,
  '\r':   1,
  ' ' :   2,
  '!' :   3,
  '"' :   4,
  '#' :   5,
  '%' :   6,
  "'" :   7,
  '(' :   8,
  ')' :   9,
  '*' :  10,
  '+' :  11,
  ',' :  12,
  '-' :  13,
  '.' :  14,
  '/' :  15,
  '0' :  16,
  '1' :  17,
  '2' :  18,
  '3' :  19,
  '4' :  20,
  '5' :  21,
  '6' :  22,
  '7' :  23,
  '8' :  24,
  '9' :  25,
  ':' :  26,
  ';' :  27,
  '<' :  28,
  '=' :  29,
  '>' :  30,
  '?' :  31,
  '@' :  32,
  'A' :  33,
  'B' :  34,
  'C' :  35,
  'D' :  36,
  'E' :  37,
  'F' :  38,
  'G' :  39,
  'H' :  40,
  'I' :  41,
  'J' :  42,
  'K' :  43,
  'L' :  44,
  'M' :  45,
  'N' :  46,
  'O' :  47,
  'P' :  48,
  'Q' :  49,
  'R' :  50,
  'S' :  51,
  'T' :  52,
  'U' :  53,
  'V' :  54,
  'W' :  55,
  'X' :  56,
  'Y' :  57,
  '[' :  58,
  ']' :  59,
  '_' :  60,
  'a' :  61,
  'b' :  62,
  'c' :  63,
  'd' :  64,
  'e' :  65,
  'f' :  66,
  'g' :  67,
  'h' :  68,
  'i' :  69,
  'j' :  70,
  'k' :  71,
  'l' :  72,
  'm' :  73,
  'n' :  74,
  'o' :  75,
  'p' :  76,

In [4]:
text_as_int = np.array([char2idx[c] for c in text])

In [5]:
print ('texto: {}'.format(repr(text[:50])))
print ('{}'.format(repr(text_as_int[:50])))

texto: 'Prologo\r\nEn 1953, Isaac Asimov publico Segunda Fun'
array([48, 78, 75, 72, 75, 67, 75,  1,  0, 37, 74,  2, 17, 25, 21, 19, 12,
        2, 41, 79, 61, 61, 63,  2, 33, 79, 69, 73, 75, 82,  2, 76, 81, 62,
       72, 69, 63, 75,  2, 51, 65, 67, 81, 74, 64, 61,  2, 38, 81, 74])


### Preparar los datos para entrenar la RNN


In [6]:
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

seq_length = 100
 
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)



In [7]:
for item in sequences.take(10):
  print(repr(''.join(idx2char[item.numpy()])))

'Prologo\r\nEn 1953, Isaac Asimov publico Segunda Fundacion, el tercer libro de la saga de la Fundacion '
'(o el decimotercero segun otras fuentes, este es un tema de debate). En Segunda Fundacion aparece por'
' primera vez Arkady Darell, uno de los principales personajes de la parte final de la saga. En su pri'
'mera escena, Arkady, que tiene 14 anos, esta haciendo sus tareas escolares. En concreto, una redaccio'
'n que lleva por titulo ?El Futuro del Plan Sheldon?. Para hacer la redaccion, Arkady esta utilizando '
'un ?transcriptor?,un dispositivo que convierte su voz en palabras escritas. Este tipo de dispositivo,'
' que para Isaac Asimov era ciencia ficcion en 1953, lo tenemos al alcance de la mano en la mayoria de'
' nuestros smartphones, y el Deep Learning es uno de los responsables de que ya tengamos este tipo de '
'aplicaciones, siendo la tecnologia otro de ellos.En la actualidad disponemos de GPUs (Graphics Proces'
'sor Units), que solo cuestan alrededor de 100 euros, que esta

In [8]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)


In [9]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'Prologo\r\nEn 1953, Isaac Asimov publico Segunda Fundacion, el tercer libro de la saga de la Fundacion'
Target data: 'rologo\r\nEn 1953, Isaac Asimov publico Segunda Fundacion, el tercer libro de la saga de la Fundacion '


In [10]:
print (dataset)

<MapDataset shapes: ((100,), (100,)), types: (tf.int64, tf.int64)>


In [11]:
BATCH_SIZE = 64

BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

print (dataset)

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>


### Construcción del modelo

In [12]:
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

In [13]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [14]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           23552     
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          5246976   
_________________________________________________________________
dense (Dense)                (64, None, 92)            94300     
Total params: 5,364,828
Trainable params: 5,364,828
Non-trainable params: 0
_________________________________________________________________


In [16]:
for input_example_batch, target_example_batch in dataset.take(1):
  print("Input:", input_example_batch.shape, "# (batch_size, sequence_length)")
  print("Target:", target_example_batch.shape, "# (batch_size, sequence_length)")


Input: (64, 100) # (batch_size, sequence_length)
Target: (64, 100) # (batch_size, sequence_length)


In [17]:
for input_example_batch, target_example_batch in dataset.take(1):  
    example_batch_predictions = model(input_example_batch)
    print("Prediction: ", example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

Prediction:  (64, 100, 92) # (batch_size, sequence_length, vocab_size)


In [18]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices_characters = tf.squeeze(sampled_indices,axis=-1).numpy()

In [19]:
sampled_indices_characters

array([37, 78, 91, 10, 58, 73, 46,  2, 87, 17, 74, 43, 36, 80, 35, 23, 52,
       21,  5, 20, 71, 62, 24, 65, 29, 28,  9, 55,  0, 49, 47, 33, 65, 14,
       40, 35, 55, 10, 29,  2,  4, 17,  5, 79, 87, 74, 81, 43, 64, 68,  1,
       52,  2, 40, 64, 14, 20, 10, 82, 23, 59, 48, 86, 21, 45, 76, 64, 30,
       54, 85, 91, 88, 12, 66, 77, 70, 91, 26, 17, 28, 58, 21, 72, 50, 32,
       38, 44,  8, 52, 70, 61,  4, 79, 50, 43, 38, 19, 79, 58, 88])

Entrenar el modelo

In [20]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [21]:
model.compile(optimizer='adam', loss=loss)

Configurar el *checkpoints*

In [22]:
 # directorio
checkpoint_dir = './training_checkpoints'
# nombre fichero
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

*Training*

In [23]:
EPOCHS=50
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/50
 8/31 [======>.......................] - ETA: 48s - loss: 3.6871

KeyboardInterrupt: 

### Generación de texto

In [ ]:
tf.train.latest_checkpoint(checkpoint_dir)

In [ ]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [ ]:
def generate_text(model, start_string):

  num_generate = 500
  input_eval = [char2idx[s] for s in start_string]

  input_eval = tf.expand_dims(input_eval, 0)
  text_generated = []


  temperature = 0.5

  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      
      predictions = tf.squeeze(predictions, 0)

      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()


      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [ ]:
print(generate_text(model, start_string=u"Alcohol "))

In [ ]:
print(generate_text(model, start_string=u"entrenar un modelo"))

In [ ]:
print(generate_text(model, start_string=u"un modelo es"))